In [1]:
%pylab inline
import tensorflow as tf
import struct, gzip
import Image
from tensorflow.examples.tutorials.mnist import input_data

Populating the interactive namespace from numpy and matplotlib


In [ ]:
# Parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])